# Difference between `kernel` and `channel`

In the context of Convolutional Neural Networks (CNNs), the terms "kernel" and "channel" refer to different components and concepts:

## 1. Kernel (also known as a Filter)
* **Definition**: A Kernek in a CNN is a small matrix used to apply convolution operations to the input data. It moves over the input data, performing element-wise multiplication followed by a summation of the results, to produce a feature map.
* **Purpose**: The purpose of a kernel is to extract specific features from the data. For example, in image processing, different kernels can detect edges, corners, textures, or other visual patterns. Each kernel is trained to become sensitive to particular aspects of the input data.
* **Dimensions**: Kernels have dimensions that are usually much smaller than the input data. For example, common kernel siezes are 3\*3, 5\*5, or 7\*7, which slide over the input image or feature map layer.
* **Multiple Kernels**: A CNN typically uses multiple kernels at each layer to create multiple feature maps. Each kernel detects different features, and the depth of the output feature map corresponds to the number of kernels used.

## 2. Channel
* **Definition**: A channel refers to the depth dimension in the input data or feature maps in CNNs. For images, typical channels are RGB, where each color (Red, Gree, Blue) has its own matrix of pixel values.
* **Role in CNNs**: Each channel in an input holds data for specific types of features or filters. In deeper layers of a CNN, each channel in the feature map can be thought of as a response from a different filter applied at the previous layer, representing various features extracted from the input.
* **Multiple Channels**: The number of channels increases as you go deeper into a CNN, where each layer can have different numbers of channels depending on the number of kernels applied in the previous layer. This growth in channels correlates with the complexity and variety of features the network learns to recognize.

## Relationship and Interaction
* When a convolution operation is performed in a CNN, each kernel is applied across all the channels of the input data or the previous layer's output (feature maps). For exmaple, if you are applying a 3\*3 kernel to an RGB image, this kernel will have aq depth of 3, and it will separately interact with each of the 3 input channels, producing a single output channel (per kernel). Thus, the kernel's operation aggregates information across channels to detect features.

In summary, kernels (filters) are the tools used to perform feature extraction and learn from the data, while channels represent the structure of the data itself, organizing different types of information or features at various stages of the network. The design of CNNs leverages the interaction between these kernels and channels to process and learn from complex data like images efficiently.

# `kernel_size` and `num_filters` in Darts' `tcn_model`

To understand the use of `tcn_model` in the Darts library, particularly the parameters `kernel_size` and `num_filters`, it's important to clarify the roles these parameters play in the model.

### `kernel_size`
- **Definition:** The `kernel_size` parameter defines the size (length) of the filter applied during the convolution operation. In the context of 1D convolutions, this refers to how many time steps each filter spans.
- **Function:** The kernel size determines the receptive field of the convolutional filter, which is the window of input data the filter looks at when computing each output value. A larger `kernel_size` allows the filter to capture patterns across a broader range of time steps, which can be beneficial for identifying longer-term dependencies in time series data.

### `num_filters`
- **Definition:** The `num_filters` parameter specifies the number of convolutional filters (also known as channels) in a layer.
- **Function:** Each filter is responsible for detecting different features or patterns in the input data. More filters allow the model to learn a richer set of features, increasing the model's capacity to capture various aspects of the input data. For instance, with `num_filters=64`, there will be 64 different filters, each producing its own feature map.

### Difference Between Kernel and Filter
- **Kernel:** Refers to the size or shape of the convolutional window that slides over the input data. It defines the local region of the input that the convolution operation is applied to at each step.
- **Filter:** Refers to the individual convolutional operations applied to the input data. Each filter produces one feature map, and multiple filters enable the extraction of different features from the same input data.

### Example Usage in Darts TCN Model

Here’s an example of how you might configure and use the TCN model in the Darts library:

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import TCNModel
from darts.metrics import mape

# Generate synthetic stock price data for trading days only
np.random.seed(42)
dates = pd.bdate_range(start='2020-01-01', end='2023-12-31')  # 'B' for business days
n = len(dates)
price = np.cumsum(np.random.normal(0, 1, n)) + 100  # Cumulative sum to simulate stock prices
data = pd.DataFrame({'Date': dates, 'Price': price})

# Ensure there are no missing dates and set the freq attribute
data.set_index('Date', inplace=True)
data = data.asfreq('B')

# Plot the synthetic data
plt.figure(figsize=(10, 6))
plt.plot(data.index, data['Price'])
plt.title('Synthetic Stock Price Data')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

# Convert to TimeSeries object
series = TimeSeries.from_dataframe(data, fill_missing_dates=False)

# Split the data into training and validation sets
train, val = series.split_before(0.8)

# Initialize the TCN model
model = TCNModel(
    input_chunk_length=30,   # Length of the input sequence
    output_chunk_length=7,   # Length of the output sequence (prediction horizon)
    kernel_size=3,           # Size of the convolutional kernel
    num_filters=16,          # Number of filters (output channels) in the convolutional layers
    dropout=0.1,             # Dropout rate
    random_state=42          # Seed for reproducibility
)

# Fit the model
model.fit(train)

# Make predictions
pred = model.predict(len(val))

# Plot the predictions
plt.figure(figsize=(10, 6))
series.plot(label='Actual')
pred.plot(label='Predicted')
plt.legend()
plt.title('TCN Model - Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

# Evaluate the model
print(f'MAPE: {mape(val, pred):.2f}%')
```

### Summary
- **`kernel_size`:** Controls the window size of the filter, influencing the range of data points the filter processes at each step.
- **`num_filters`:** Determines the number of different filters applied, each capturing different features from the input data.

These parameters work together to shape the capacity and the granularity of the features the TCN model can learn from the time series data. Adjusting these parameters can help optimize the model for different types of time series patterns and complexities.